In [2]:
from tensorflow.keras.preprocessing import sequence
import tensorflow as tf
import keras
import os
import numpy as np

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


reading content of file

In [4]:
text=open(path_to_file,'rb').read().decode(encoding='utf-8')
print('length of text :{} characters'.format(len(text)))

length of text :1115394 characters


In [5]:
#encoding
vocab=sorted(set(text))
#creating a mapping  from unique characters to indices
char2indx={u:i for i,u in enumerate(vocab)}
indx2char=np.array(vocab)
def text_to_int(text):
  return np.array([char2indx[c] for c in text])
text_as_int=text_to_int(text)

In [6]:
#lets look part of our encoded text
print("text: ",text[:13])
print("encoded: ",text_to_int(text[:13]))

text:  First Citizen
encoded:  [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
#converting integer to text
def int_to_text(ints):
  try:
    ints=ints.numpy()
  except:
    pass
  return ''.join(indx2char[ints])
print(int_to_text(text_as_int[:13]))

First Citizen


creating training example

In [8]:
seq_length=100 #length of sequence for training example
example_per_epoch=len(text)//(seq_length+1)
#creating training examples/targets
char_datasets=tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences=char_datasets.batch(seq_length+1, drop_remainder=True)

In [10]:
#splitting seqlen 101  into inout and output
def split_input_target(chunk):
  input_text=chunk[:-1]#hell
  target_text=chunk[1:]#ello
  return input_text,target_text #hell, ello
dataset=sequences.map(split_input_target)

In [11]:
for x,y in dataset.take(2):
  print("\n\nExample\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT\n")
  print(int_to_text(y))



Example

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT

irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


Example

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT

re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [12]:
#making training data
BATCH_SIZE=64
VOCAB_SIZE=len(vocab)
EMBEDDING_DIM=256
RNN_UNITS=1024
#BUFFER SIZE TO SUFFLE THE DATASET
BUFFER_SIZE=10000
data=dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE,drop_remainder=True)

building model

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = keras.Sequential([
        keras.layers.Embedding(vocab_size, embedding_dim),
        keras.layers.LSTM(rnn_units,
                          return_sequences=True,
                          stateful=True,
                          recurrent_initializer='glorot_uniform'),
        keras.layers.Dense(vocab_size)
    ])
    return model


# Build the model
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)

# Specify input shape manually
model.build(input_shape=(BATCH_SIZE, None))

# Print the model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, None, 256)             │          16,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (64, None, 1024)            │       5,246,976 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, None, 65)              │          66,625 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,330,241 (20.33 MB)

 Trainable params: 5,330,241 (20.33 MB)

 Non-trainable params: 0 (0.00 B)

creating loss function

In [14]:
for input_example_batch,target_example_batch in data.take(1):
  example_batch_predictions=model(input_example_batch) #ask our model for aprediction on our first batch of training data
  print(example_batch_predictions.shape,"# (batch_size, sequence_length,vocab_size)")

(64, 100, 65) # (batch_size, sequence_length,vocab_size)


In [15]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-4.47535701e-03 -6.38556620e-03  1.86693540e-03 ... -4.45628678e-03
    1.55942258e-03  3.75115289e-03]
  [-1.46600278e-03  9.61179467e-05  1.53545104e-03 ... -9.19833966e-03
   -6.38451194e-04  3.54639557e-03]
  [-4.25120303e-03  7.60611147e-04  3.15016764e-03 ... -8.86621047e-03
   -6.10497082e-04  1.53507444e-03]
  ...
  [ 9.34165716e-03  7.75493030e-03  7.63940904e-03 ...  5.20302774e-03
   -8.15550983e-03 -3.30788339e-03]
  [ 5.30312443e-03  8.16131476e-03  9.17068869e-03 ...  1.13695720e-02
   -5.94360335e-03 -5.01992356e-04]
  [ 4.59625665e-03  7.67902751e-03  8.98114964e-03 ...  1.09219104e-02
   -7.06301117e-03 -3.59634473e-03]]

 [[ 4.09850836e-05 -5.51115302e-03 -8.93503334e-03 ...  1.34692586e-03
    5.93029708e-03  4.96851513e-04]
  [ 3.42853367e-03 -2.82472954e-03 -8.28641374e-03 ... -3.39888153e-04
    1.15102557e-02 -2.81255948e-03]
  [ 4.77665849e-03 -4.45961545e-04 -1.19157284e-02 ...  8.60534128e-05
    7.32537918e-03 -6.11716742e-03]
  ...
  [ 3.157

In [16]:
#examine one prediction
pred=example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-4.4753570e-03 -6.3855662e-03  1.8669354e-03 ... -4.4562868e-03
   1.5594226e-03  3.7511529e-03]
 [-1.4660028e-03  9.6117947e-05  1.5354510e-03 ... -9.1983397e-03
  -6.3845119e-04  3.5463956e-03]
 [-4.2512030e-03  7.6061115e-04  3.1501676e-03 ... -8.8662105e-03
  -6.1049708e-04  1.5350744e-03]
 ...
 [ 9.3416572e-03  7.7549303e-03  7.6394090e-03 ...  5.2030277e-03
  -8.1555098e-03 -3.3078834e-03]
 [ 5.3031244e-03  8.1613148e-03  9.1706887e-03 ...  1.1369572e-02
  -5.9436033e-03 -5.0199236e-04]
 [ 4.5962567e-03  7.6790275e-03  8.9811496e-03 ...  1.0921910e-02
  -7.0630112e-03 -3.5963447e-03]], shape=(100, 65), dtype=float32)


In [17]:
#prediction at the first timestamp
time_pred=pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-4.4753570e-03 -6.3855662e-03  1.8669354e-03 -8.9455361e-04
  4.7953320e-03  6.0206209e-03  4.0617273e-03 -2.3540817e-03
 -1.8198616e-03  2.5351212e-04 -1.8807746e-03  8.2353008e-04
  7.5220771e-04 -4.3601114e-03 -3.8000106e-04  3.1382022e-03
 -3.6413099e-03 -5.5203843e-03 -3.5231023e-03 -1.9056926e-04
  4.5033335e-03  6.3881319e-04 -1.2329571e-03 -4.5609223e-03
 -4.7779168e-04 -5.4198229e-03 -6.4648599e-03  1.1239626e-03
  9.0522564e-04  2.2085740e-03 -2.5423762e-04 -2.4571291e-03
 -4.2497762e-03  1.3020060e-03 -1.3874591e-03 -2.1951615e-03
 -3.9154198e-03  1.5499425e-03 -3.5458670e-03  2.0686151e-03
 -2.2978901e-03 -3.2178010e-03 -4.8027290e-03 -7.6828431e-04
 -4.9587125e-03 -2.0162040e-06  4.9274536e-03 -7.6648891e-03
 -1.4188200e-03  1.6657751e-03 -3.9306539e-03 -1.7531289e-04
 -3.1853386e-03 -8.8487455e-04 -3.4061600e-03  4.6583735e-03
  2.4065222e-03  3.5473227e-03 -5.1855766e-03 -2.5587871e-03
  1.3284090e-03 -7.3565857e-04 -4.4562868e-03  1.5594226e-03
  3.751152

if we want to determine the predicted character we need to sample the output distribution ie picking a value based on probability

In [18]:
sample_indices=tf.random.categorical(pred,num_samples=1)
#now we can reshape that array and convert integers to number to see character
sampled_indices=np.reshape(sample_indices,(1,-1))[0]
predicted_char=int_to_text(sampled_indices)
predicted_char

"tc,aOxvzjyAK\nC.a!MkL:rK&M&k:$wgGN'LvRWX.rXXKS-Vvw$q!YeYfPxs$e$vrYJqzntCZ;CTBpAzN?&p'pJNmvB'cdpPN'iev"

In [19]:
def loss(labels,logits):
  return keras.losses.sparse_categorical_crossentropy(labels,logits,from_logits=True)

In [20]:
#compiling the model
model.compile(optimizer='adam',loss=loss)

In [21]:
#creating checkpoint
#dir where checkpoint will be saved
checkpoint_dir='./training_checkpoint'
#check point name
checkpoint_prefix=os.path.join(checkpoint_dir,"ckpt_{epoch}.weights.h5")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

training

In [22]:
history=model.fit(data,epochs=40,callbacks=[checkpoint_callback])

Epoch 1/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 64ms/step - loss: 2.8635
Epoch 2/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - loss: 1.8580
Epoch 3/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - loss: 1.6059
Epoch 4/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 66ms/step - loss: 1.4844
Epoch 5/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 13s 67ms/step - loss: 1.4058
Epoch 6/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 21s 69ms/step - loss: 1.3561
Epoch 7/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 69ms/step - loss: 1.3154
Epoch 8/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 14s 69ms/step - loss: 1.2787
Epoch 9/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - loss: 1.2423
Epoch 10/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 15s 71ms/step - loss: 1.2111
Epoch 11/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 71ms/step - loss: 1.1787
Epoch 12/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 20s 71ms/step - loss: 1.1510
Epoch 13/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 22s 72ms/step - loss: 1.1173
Epoch 14/40
172/172 ━━━━━━━━━━━━━━━━━━━━ 19s 71ms/step - loss: 1.0844
Epoch 15/40
172/172 ━━━━━━━━━

In [52]:

model=build_model(VOCAB_SIZE,EMBEDDING_DIM,RNN_UNITS,batch_size=1)

model.build(input_shape=(1,None))
model.load_weights('./training_checkpoint/ckpt_40.weights.h5')
model.build(input_shape=(1,None))

In [24]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
print(latest_checkpoint)


None


In [55]:
#model.build(tf.TensorShape([1,None]))
model.load_weights('./training_checkpoint/ckpt_4.weights.h5')
model.build(tf.TensorShape([1,None]))

In [72]:
#loading specific checkpoint
checkpoint_num=40
model.load_weights("./training_checkpoint/ckpt_"+str(checkpoint_num)+".weights.h5")
model.build(tf.TensorShape([1,None]))

In [73]:
#generating text
def generate_text(model,start_string):

  #num of character to generate
  num_generate=400
  input_eval=[char2indx[s] for s in start_string]
  input_eval=tf.expand_dims(input_eval,0)

  text_generated=[]
  #low temp= more predictable text
  #high temp=more surprising text
  temperature=1.2
  #here batch_size =1
  #model.reset_states()
  for i in range(num_generate):
    predictions=model(input_eval)
    predictions=predictions[:,-1,:]
    predictions=predictions/temperature
    predicted_id=tf.random.categorical(predictions,num_samples=1)[-1,0].numpy()

    input_eval=tf.expand_dims([predicted_id],0)
    text_generated.append(indx2char[predicted_id])
  return(start_string+"".join(text_generated))

In [75]:
inp=input("type a starting string:")
print(generate_text(model,inp))

type a starting string:ISABELLA
ISABELLA:
Grands of thing.

PERDITA:
O slave!
Why that's some coy.
Now, then indeed the better, the mouty testamony
That crew caps that our chances with my wounds, and from the father,
Stand you my five reasons well encounter'd,
Which should bequice me to question
Are you most Ducy find straight long hath touch'd upon't:
Therefore thy mind come near my soul's curse,
Lest Isabellare,
No more words it gains
